### Další koncepty objektově-orientovaného programování
---

<br>

#### Podtržítka v Pythonu

---
Podtržítko je **syntaktický znak** v Pythonu, který má nejeden význam pro Python:
1. Přeskoč `_`,
2. privátní proměnná  `_name`,
3. chráněná proměnná `__name`,
4. přepis klíčové slovo `class_`,
5. magická metoda `__init__`.

<br>

##### Samotné podtržítko (*skip*)

In [ ]:
import time

def check_logging_messages(limit: int) -> None:
    """Check the current logging messages."""
    
    for sec in range(limit):
        print("Checking logging messages..")
        time.sleep(1)
        
check_logging_messages(5)

<br>

Proměnná `sec`, kterou jsme vytvořili v rámci funkce `check_logging_messages` nemá žádné využití.

<br>

Proto v Pythonu existuje symbol `_` pro označení **nepoužívané** proměnné:
```python
    for _ in range(limit):
        print("Checking logging messages..")
```

<br>

Případně je možné **přeskakovat** i u *vícenásobného* přiřazování:
```python
jmeno, domena = "matous@gmail.com".split("@")

# nepotřebuji proměnnou jméno
_, domena = "matous@gmail.com".split("@")
print(domena)
```

<br>

##### Slabé privátní objekty (*weak private*)

Některé jazyky umožňují práci s pomocí **privátních** (soukromých) proměnných (~Java).

<br>

Python tuto funkcionalitu **nepodporuje**:

In [ ]:
import time

class ProtocolChecker:
    
    def __init__(self, limit: int, protocol: str):
        self._limit = limit
        self.protocol = protocol
        
    def check_protocol(self) -> None:
        for _ in range(self._limit):
            print(f"Checking routing protocol:{self.protocol}...")
            time.sleep(1)

In [ ]:
checker = ProtocolChecker(5, "ospf")
print(checker._limit)

In [ ]:
checker._limit = 10
print(checker._limit)

<br>

Pokud metodu nebo proměnnou označíš pomocí **jednoho podtržítka**, naznačuješ ostatním programátorům, že tento objekt je **interní** a není *DOPORUČENÉ* jej přepisovat.

<br>

##### Chráněné proměnné (*strong private*)

Pomocí **dvou podtržítek** může uživatel definovat **chráněné** proměnné, a tím předejít neúmyslnému přepsání nebo přetypování:

In [1]:
import time

class ProtocolChecker:
    
    def __init__(self, limit: int, protocol: str):
        self._limit = limit
        self.protocol = protocol
        
    def check_protocol(self) -> None:
        for _ in range(self._limit):
            print(f"Checking routing protocol:{self.protocol}...")
            time.sleep(1)

In [2]:
checker = ProtocolChecker(5, "bgp")
print(f"{checker.__limit=}")

AttributeError: 'ProtocolChecker' object has no attribute '__limit'

In [3]:
print(checker.__dict__)

{'_ProtocolChecker__limit': 5}


In [5]:
checker._ProtocolChecker__limit = 10
print(checker.__dict__)

{'_ProtocolChecker__limit': 10}


<br>

Nicméně ani toto řešení není 100%. Pomocí metody `__dict__` je možné zjistit, jaké proměnné (..a hodnoty) má instance k dispozici a dohledáme přejmenovanou proměnnou.

<br>

##### Vlastní jméno (*any*)

Pokud se ti bude krýt **klíčové slovo** se jménem proměnné, interpret ti bude vracet syntaktickou výjimku:

In [10]:
class Employer:
    def __init__(self, name: str, email: str):
        self.name = name
        self.email = email

class = Employer("Matous", "matous@gmail.com")

SyntaxError: invalid syntax (<ipython-input-10-9e5025b3f93d>, line 6)

<br>

Pokud tomu chceš zabránit, můžeš použít podtržítko jako příponu za klíčovým výrazem:
```python
class_ = Employer("Matous", "matous@gmail.com")
```

<br>

##### Magické metody (double-underscore methods ~ dunder methods)

Magické metody jsou **speciální metody** v Pythonu, na kterých stojí veškerá práce s objekty.

<br>

Oficiální dokumentace ale není příliš nápomocná a jejich používání není úplně nejčastější (výjimky typu `__init__` aj.).

<br>

#### Vlastnosti třídy

---
Jak tedy používat proměnné s podtržítky? Kde je použít, kde použít obyčejné proměnné?

<br>

Představ si situaci, kdy máš napsat převodník jednotek objemu. Z **litrů** na **pinty**(UK):

In [1]:
class LiterConvertor:
    def __init__(self, liter: int = 0):
        self.liter = liter
        self.pint_uk_coef: float = 1.759754
        self.pints_uk = self.liter * self.pint_uk_coef

In [2]:
bottle_volume = LiterConvertor()
bottle_volume.liter = 0.75
print(f"{bottle_volume.liter=}")

bottle_volume.liter=0.75


In [3]:
print(f"{bottle_volume.pints_uk=}")

bottle_volume.pints_uk=0.0


<br>

Kdykoliv se pokusíš tímto způsobem přepsat hodnotu, interpret pracuje stejně jako když přepisuješ hodnotu ve slovníku:

In [40]:
 print(bottle_volume.__dict__)

{'liter': 0.75, 'pint_uk_coef': 1.759754, 'pints_uk': 0.0}


In [6]:
print(
    f"{bottle_volume.liter=}",              # instační atribut
    f"{bottle_volume.__dict__['liter']=}",  # slovníkový výběr podle klíče
    sep="\n"
)

bottle_volume.liter=0.75
bottle_volume.__dict__['liter']=0.75


<br>

Programátoři z jiných jazyků, např. Java, by napsali jiné řešení:

In [7]:
class LiterConvertor:

    def __init__(self, liter: int = 0):
        self.set_volume_in_liter(liter)
        self.pint_uk_coef: float = 1.759754
        
    def to_pints(self):
        return self._liter * self.pint_uk_coef
    
    def get_volume_in_liter(self):
        return self._liter
    
    def set_volume_in_liter(self, value: float):
        if value < 0:
            raise ValueError("Cannot process negative number")
        self._liter = value  # privátní proměnná

In [8]:
volume = LiterConvertor(0.75)
print(
    f"{volume.get_volume_in_liter()=}",  # pro hodnotu musím použít metodu
    f"{volume.to_pints()=}",
    sep="\n"
)

volume.get_volume_in_liter()=0.75
volume.to_pints()=1.3198155


In [9]:
volume.set_volume_in_liter(-1)

ValueError: Cannot process negative number

<br>

V tento moment reálně pracujeme s privátní proměnnou, tak jak v jiných jazycích.

<br>

Náš zápis má umožnit efektivní přepsání hodnotyve stávajícím zápisu, ale v tento moment bychom provedli příliš mnoho zásahů do původního zápisu.

<br>

Bohužel se také objevil problém s ohledem na implementaci našeho vylepšení. Všechny ohlášení `obj.liter` je nutné přepsat na `obj.get_volume_in_liter()` a `obj.liter = val` na `obj.set_volume(val)`.

<br>

Taková úprava řešení může znamenat problém na desítky, stovky řádků, protože přidáme nové metody, které je potřeba použít.

<br>

Naštěstí můžeme s vlastnostmi třídy nakládat lépe pomocí funkce `property`:

In [11]:
class LiterConvertor:
    def __init__(self, liter: float = 0.0):
        self.liter = liter
        self.pint_uk_coef: float = 1.759754
        
    def to_pints(self):
        return self._liter * self.pint_uk_coef
    
    def get_volume_in_liter(self):
        print("Getting..")
        return self._liter
    
    def set_volume_in_liter(self, value: float):
        print("Setting..")
        if value < 0:
            raise ValueError("Cannot process negative number")
        self._liter = value
        
    liter = property(get_volume_in_liter, set_volume_in_liter)

In [12]:
volume = LiterConvertor(0.75)
print(
    f"{volume.liter=}",  # opět stačí aplikovat jen atribut metody (privátní atribut)
    f"{volume.to_pints()=}",
    sep="\n"
)

Setting..
Getting..
volume.liter=0.75
volume.to_pints()=1.3198155


In [55]:
volume.liter = -1

Setting..


ValueError: Cannot process negative number

<br>

Pokaždé co tentokrát použijeme proměnnou `liter` dojde automaticky k zavolání metody `get_volume_in_liter()`.

<br>

Stejně tak, pokud budeš chtít přepast hodnotu v proměnnou `liter`, tak dojde ke spuštění metody `set_volume_in_liter()`.

<br>

Takže pokud použiješ funkci `property()`, nemusíš se omezovat na potřebné úpravy tvého stávajícího skriptu.

<br>


#### Dekorátory setter, getter, deleter

---

<br>

Zabudovaná funkce `property` při spuštění vytváří objekt `property`. V dokumentaci uvidíš syntaxi:
```python
property(fget=None, fset=None, fdel=None, doc=None)
```

<br>

V předchozí ukázce jsme zapisovali ručně metody `set_volume` a `get_volume`, které odpovídají volitelným argumentém u funkce `property`.

<br>

Zkušenější programátoři s těmito atributy pracují pomocí **dekorátorů** `@property` a `@setter`.

<br>

Pokud **neznáš dekorátory**, nic si z toho nedělej. Pro skutečnou implementaci tohoto řešení stačí pracovat s jejich abstrakcemi.

<br>

Původní zápis pomocí dekorátorů upravíme následovně:

In [56]:
class LiterConvertor:
    def __init__(self, liter: float = 0.0):
        self.liter = liter
        self.pint_uk_coef: float = 1.759754
        
    def to_pints(self):
        return self._liter * self.pint_uk_coef
    
    @property         # označíme jako vlastnost pomocí dekorátoru
    def liter(self):  # jméno metody nezačíná slovesem
        print("Getting..")
        return self._liter
    
    @liter.setter     # dekorátor pro nastavení hodnoty 
    def liter(self, value: float):  # stejnojmenná metoda
        print("Setting..")
        if value < 0:
            raise ValueError("Cannot process negative number")
        self._liter = value

In [57]:
volume = LiterConvertor(0.75)
print(volume.liter)
print(volume.to_pints())

Setting..
Getting..
0.75
1.3198155


In [7]:
volume.liter = -1

Setting..


ValueError: Cannot process negative number

<br>

Dnes jde o nejrozšířenější implementaci nastavení **vlastostni třídy**, která vyplývá z funkce `property`. Výsledné řešení je poměrně jednoduché, přehledné a dá se pochopit bez podrobné znalosti dekorátorů.

<br>

#### Metody v OOP

---

Jako ucelený seznam metod, které můžeme použít v rámci OOP se podívej na tyto metody:
1. **Instanční** metoda,
2. **třídní** metoda,
3. **statická** metoda.

<br>

Celou situaci lze vysvětlit na teoretické ukázce:

In [ ]:
class KindleNotesParser:
    """Parse the data in the .txt file"""
    
    def load_data(self):     # klasická metoda instance
        return "Calling instance method..", self
    
    @classmethod
    def parsing_files(cls):  # třídní metoda
        return "Calling class method..", cls
    
    @staticmethod
    def is_there_file():     # statická metoda
        return "Calling static method.."

In [ ]:
reader = KindleNotesParser()

<br>

##### Instační metoda
Metoda, která má mj. mezi parametry klíčový výraz `self`. Tím dovede zpřístupnit jak třídy (a její atributy), tak instance.

In [ ]:
print(
    reader,              # adresa objektu instance 'reader'
    reader.load_data,    # popis metody patřící instanci 'reader'
    reader.load_data(),  # spuštění, opět adresa
    sep="\n"
)

<br>

Takže pomocí předchozí ukázky můžeme říct, že **instanční metoda** je schopna zpřístupnit *původní třídu* a případně pracovat s jejími *atributy*.

<br>

##### Třídní metoda
Podobný zápis jako pro instační metodu, ale liší se dvěma zásadním rozdíly:
1. Dekorátor `@classmethod`,
2. parametr `cls`.

In [ ]:
print(
    reader.parsing_files,    # adresa objektu instance nyní chybí
    reader.parsing_files(),  # metodu spouští, třídy zpřístupní, ale instanci nezná
    sep="\n"
)

<br>

Jakmile použiješ **třídní metodu**, vidíš že máš přístup k objektu *původní třídy*, ale tentokrát není k dispozici odkaz (adresa) *instance*.

```
('Reading .txt file..', <__main__.KindleParser object at 0x7f0ed30782e0>)
('Spoustim metodu tridy', <class '__main__.KindleParser'>)
```

<br>

##### Statická metoda

1. Dekorátor `@staticmethod`,
2. chybí parametr `self`,
3. chybí parametr `cls`.

In [ ]:
print(
    reader.is_there_file,
    reader.is_there_file(),
    sep="\n"
)

In [ ]:
print(KindleNotesParser.load_data())

In [ ]:
print(KindleNotesParser.parsing_files())

In [ ]:
print(KindleNotesParser.is_there_file())

<br>

Pokud spustíš a prozkoumáš **statickou metodu**, můžeš si ověřit, že tato metoda nemá přístup ani k *původní třídě*, ani k její *instanci*.

<br>

##### Shrnutí k metodám

---
1. **instanční metoda** - může upravit nejenom objekty instance, ale i třídy (na začátku vidí jak třídu, tak instanci),
2. **třídní metoda (@classmethod)** - může upravit objekty třídy, ale nemůže upravovat objekty instancí (vidí třídu, ale ne instanci),
3. **statická metoda (@staticmethod)** - nemůže upravovat ani objekty instancí, ani objekty třídy (nevidí ani třídu, ani instanci).

<br>

##### Praktické ukázky

---

###### Instanční metoda

In [ ]:
class KindleNotesParser:
    """Parse the data in the .txt file"""
    notes: list = []  # upravení třídního atributu
    
    def __init__(self, file: str, data: str):
        self.file = file
        self.data = data
#         self.notes = list()  # upravení instan. atributu
        
    def load_data(self):
        return self.notes.append(self.data)

In [ ]:
first_note = KindleNotesParser("poznamky.txt", "Moje první poznámka k ...")
second_note = KindleNotesParser("poznamky_nove.txt", "Druhá poznámka ke knížce ...")

first_note.load_data()
second_note.load_data()

print(first_note.notes, second_note.notes, sep="\n")

<br>

Je jedno, který atribut budeš chtít upravit. Díky **instanční metodě** můžeš pracovat jak s třídními, tak s instančními objekty.
<br>

###### Třídní metoda

In [ ]:
class KindleNotesParser:
    """Parse the data in the .txt file"""
    readed_files: int = 0
    
    def __init__(self, file: str):
        self.file = file
    
    @classmethod
    def parsing_files(cls, name):
        instance = cls(name)
        cls.readed_files += 1  
        print(f"Parsing data from: {name}")

In [ ]:
print(f"{KindleNotesParser.readed_files=}")

In [ ]:
KindleNotesParser.parsing_files("poznamky.txt")
KindleNotesParser.parsing_files("nove_poznamky.txt")
KindleNotesParser.parsing_files("poznpozn.txt")

In [ ]:
print(f"{KindleNotesParser.readed_files=}")

<br>

Pokud budeš chtít použít **třídní metody**, potom dávej pozor na to, že můžeš spravovat pouze třídní atributy.
<br>

###### Statická metoda

In [ ]:
import os

class KindleNotesParser:
    """Parse the data in the .txt file"""
    
    def __init__(self, file: str):
        self.file = file
    
    @staticmethod
    def is_there_file(name):
        print("The file exists!") if os.path.exists(name) else print("Does not exist!")


In [ ]:
parser = KindleNotesParser("")
parser.is_there_file("lesson01.ipynb")

In [ ]:
parser.is_there_file("lesson11.ipynb")

<br>

**Statická metoda** nepotřebuje vědět nic ani o třídě, ani o instanci. Pracuje s parametrem jako klasická funkce. Ale svým účelem spadá jako nástroj ke konkrétní třídě.

<br>

#### Úloha
----

První úlohou bude napsat službu zpracovávající firemní tickety.

Průběh celé aplikace:
```
"""
1. Iniciace aplikace (zadáme škálu důležitosti priorit),
2. zadáme několik nových ticketů (jméno, zpráva),
3. zpracujeme tickety,
"""
```


Následný výstup:
```
Created: 09/11/21, 08:37:09
Processing ticket id: TPTUZSIY
Issue: ...
Customer: ...
Importance: 9
==================================
Created: 09/11/21, 08:37:09
Processing ticket id: DOGGOKZL
Issue: ...
Customer: ...
Importance: 10
==================================
Created: 09/11/21, 08:37:09
Processing ticket id: LUTAIXAS
Issue: ...
Customer: ...
Importance: 4
==================================
```

In [13]:
note = \
"""
==========
Faktomluva (Hans Rosling;Ola Rosling;Anna Roslingová Rönnlundová)
- Your Highlight on Location 2724-2728 | Added on Wednesday, July 24, 2019 8:41:27 AM

Největším orgánem našeho těla je kůže. Před objevem moderních léků patřila k nejhorším kožním nemocem syfilis. Začínala jako svědivé vřídky a pak si prokousala cestu do kostí, až postihla celou kostru. Nemoc způsobující ohavný vzhled a nesnesitelnou bolest měla v různých zemích různá jména. V Rusku chorobě říkali „polská nemoc“. V Polsku to byla „německá nemoc“, v Německu „francouzská nemoc“ a ve Francii „italská nemoc“. Italové vinu házeli zpátky a nazývali ji „francouzská nemoc“.
==========

"""

In [19]:
note_atributes = note.splitlines()[2:-2]; note_atributes

['Faktomluva (Hans Rosling;Ola Rosling;Anna Roslingová Rönnlundová)',
 '- Your Highlight on Location 2724-2728 | Added on Wednesday, July 24, 2019 8:41:27 AM',
 '',
 'Největším orgánem našeho těla je kůže. Před objevem moderních léků patřila k\xa0nejhorším kožním nemocem syfilis. Začínala jako svědivé vřídky a\xa0pak si prokousala cestu do kostí, až postihla celou kostru. Nemoc způsobující ohavný vzhled a\xa0nesnesitelnou bolest měla v\xa0různých zemích různá jména. V\xa0Rusku chorobě říkali „polská nemoc“. V\xa0Polsku to byla „německá nemoc“, v\xa0Německu „francouzská nemoc“ a\xa0ve Francii „italská nemoc“. Italové vinu házeli zpátky a\xa0nazývali ji „francouzská nemoc“.']

In [24]:
tuple(filter(None, note_atributes))

('Faktomluva (Hans Rosling;Ola Rosling;Anna Roslingová Rönnlundová)',
 '- Your Highlight on Location 2724-2728 | Added on Wednesday, July 24, 2019 8:41:27 AM',
 'Největším orgánem našeho těla je kůže. Před objevem moderních léků patřila k\xa0nejhorším kožním nemocem syfilis. Začínala jako svědivé vřídky a\xa0pak si prokousala cestu do kostí, až postihla celou kostru. Nemoc způsobující ohavný vzhled a\xa0nesnesitelnou bolest měla v\xa0různých zemích různá jména. V\xa0Rusku chorobě říkali „polská nemoc“. V\xa0Polsku to byla „německá nemoc“, v\xa0Německu „francouzská nemoc“ a\xa0ve Francii „italská nemoc“. Italové vinu házeli zpátky a\xa0nazývali ji „francouzská nemoc“.')

In [20]:
note = dict()

note["name"] = note_atributes[0]
note["location"] = note_atributes[1].split("|")[0]
note["date"] = note_atributes[1].split("|")[1]
note["description"] = note_atributes[-1]

In [22]:
from pprint import pprint
pprint(note)

{'date': ' Added on Wednesday, July 24, 2019 8:41:27 AM',
 'description': 'Největším orgánem našeho těla je kůže. Před objevem moderních '
                'léků patřila k\xa0nejhorším kožním nemocem syfilis. Začínala '
                'jako svědivé vřídky a\xa0pak si prokousala cestu do kostí, až '
                'postihla celou kostru. Nemoc způsobující ohavný vzhled a\xa0'
                'nesnesitelnou bolest měla v\xa0různých zemích různá jména. '
                'V\xa0Rusku chorobě říkali „polská nemoc“. V\xa0Polsku to byla '
                '„německá nemoc“, v\xa0Německu „francouzská nemoc“ a\xa0ve '
                'Francii „italská nemoc“. Italové vinu házeli zpátky a\xa0'
                'nazývali ji „francouzská nemoc“.',
 'location': '- Your Highlight on Location 2724-2728 ',
 'name': 'Faktomluva (Hans Rosling;Ola Rosling;Anna Roslingová Rönnlundová)'}


---